In [4]:
from mlflow.tracking import MlflowClient
from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer, pipeline
import mlflow.transformers


# Input arguments
model_dir = "/mnt/data/test-fe8a4e/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/fe8a4ea1ffedaf415f4da2f062534de366a451e6"
model_id = "TinyLlama__TinyLlama-1.1B-Chat-v1.0"
revision = "fe8a4ea1ffedaf415f4da2f062534de366a451e6"
model_type = "LLM"


# Load the model and tokenizer from model_dir based on model_type
if model_type == "LLM":
    model = AutoModelForCausalLM.from_pretrained(model_dir)
else:
    model = AutoModel.from_pretrained(model_dir)
    
tokenizer = AutoTokenizer.from_pretrained(model_dir)
print("Loaded model and tokenizer")

with mlflow.start_run() as run:
    # Log the model
    mlflow.transformers.log_model(
        transformers_model={"model": model, "tokenizer": tokenizer},
        artifact_path="model",
        task="text-generation",
        input_example="Hello, how are you?",
    )
    print("Logged model")

    # TODO Check for metadata conflicts if the registered model already exists.
    # For example, the model_id must match the existing tag.

    # Register the model
    result = mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/model",
        name=model_id,
    )
    print(f"Registered model={model_id}, version={result.version}")
    
client = MlflowClient()

# First-time registration
if result.version == "1":
    # Update description
    client.update_registered_model(
        name=model_id,
        description=f"{model_id} @ {model_version} from Hugging Face"
    )
    print(f"Updated registered model description: {description}")

    # Tag model ID
    client.set_registered_model_tag(name=model_id, key="mlflow.domino.model_id", value=model_id)
    print("Tagged mlflow.domino.model_id={model_id}")

# Tag model type
client.set_registered_model_tag(name=model_id, key="mlflow.domino.model_type", value=model_type)
print(f"Tagged mlflow.domino.model_type={model_type}")
    
# Tag model version
client.set_model_version_tag(
    name=model_id,
    version=result.version,
    key="mlflow.domino.model_version",
    value=revision,
)
print("Tagged mlflow.domino.model_version={model_version}")
    

Device set to use cuda:0


Loaded model and tokenizer


2025/07/26 05:47:06 WARNING mlflow.transformers: The model card could not be retrieved from the hub due to Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/mnt/data/test-fe8a4e/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/fe8a4ea1ffedaf415f4da2f062534de366a451e6'. Use `repo_type` argument if needed.
2025/07/26 05:47:06 WARNING mlflow.transformers: Unable to find license information for this model. Please verify permissible usage for the model you are storing prior to use.
2025/07/26 05:47:06 INFO mlflow.transformers.signature: Running model prediction to infer the model output signature with a timeout of 180 seconds. You can specify a different timeout by setting the environment variable MLFLOW_INPUT_EXAMPLE_INFERENCE_TIMEOUT.
/opt/conda/lib/python3.10/site-packages/mlflow/transformers/signature.py:150: FutureWarning: ``mlflow.transformers.signature.generate_signature_output`` is deprecated since 2.19.0. This method will be removed in a future release. Use `

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

Device set to use cuda:0


Logged model


Successfully registered model 'TinyLlama--TinyLlama-1.1B-Chat-v1.0'.
2025/07/26 05:50:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: TinyLlama--TinyLlama-1.1B-Chat-v1.0, version 1
Created version '1' of model 'TinyLlama--TinyLlama-1.1B-Chat-v1.0'.


Registered model=TinyLlama--TinyLlama-1.1B-Chat-v1.0, version=1
🏃 View run mysterious-fowl-720 at: http://127.0.0.1:8768/#/experiments/17/runs/8b2dc22aa0fa47b08f6ee324501036fd
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/17
